# Langchain Agent

In [1]:
from modules.openai_agent import create_agent
from modules.chat_history import ChatHistory

In [2]:
from langchain.agents import tool
from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

# get_word_length.invoke("abc")

tools = [get_word_length, TavilySearchResults(max_results=1)]

In [3]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
chat = ChatHistory("Welcome to the chat!")
agent = create_agent(prompt=prompt, tools=tools, chat=chat)
model = create_agent(tools=tools, model_only=True, streaming=True)

In [4]:
user_input = "What is langraph?"
agent.invoke({"input": user_input, "chat_history": chat.history})

{'input': 'What is langraph?',
 'chat_history': [SystemMessage(content='Welcome to the chat!'),
  HumanMessage(content='What is langraph?'),
  AIMessage(content='LangGraph is a new package available in both Python and JavaScript that enables the creation of LLM (LangChain Language Model) workflows containing cycles, which are a critical component of most agent runtimes. It is fully integrated into the LangChain ecosystem, allowing users to take full advantage of all the LangChain integrations and LangSmith observability. LangGraph is designed to facilitate multi-agent workflows and has been used to build applications that utilize the concept of multiple agents. You can find more information about LangGraph in this [blog post](https://blog.langchain.dev/langgraph-multi-agent-workflows/).')],
 'output': 'LangGraph is a new package available in both Python and JavaScript that enables the creation of LLM (LangChain Language Model) workflows containing cycles, which are a critical component

# Langraph

In [5]:
from langgraph.prebuilt import ToolInvocation, ToolExecutor
from langchain_core.messages import ToolMessage
import json
from modules.create_graph import create_graph, stream_app


tool_executor = ToolExecutor(tools)

tool_indicator = "tool_calls" #"function_call"

# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if tool_indicator not in last_message.additional_kwargs:
        return "end"
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

# Define the function that calls the model
def call_agent(state):
    messages = state['messages']
    response = agent.invoke(
        {"input": messages[0], "chat_history": chat.history})
    return {"messages": [response['output']]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    last_message = messages[-1]
    
    tool = last_message.additional_kwargs[tool_indicator][0]
    action = ToolInvocation(
        tool=tool['function']["name"],
        tool_input=json.loads(tool['function']['arguments'])
    )

    response = tool_executor.invoke(action)
    function_message = ToolMessage(content=str(response), tool_call_id=tool['id'])
    return {"messages": [function_message]}


nodes = [{"name": "agent", "node": call_model}, {
    "name": "action", "node": call_tool, "condition": should_continue}]

app = create_graph(nodes)

user_input = "what is the weather in sf"
stream_app(app, user_input)

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_I5iCBbxbAftTyFq7B4v8IZ1j', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]})]}

---

Output from node 'action':
---
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "Weather in San Francisco is {\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1708570265, \'localtime\': \'2024-02-21 18:51\'}, \'current\': {\'last_updated_epoch\': 1708569900, \'last_updated\': \'2024-02-21 18:45\', \'temp_c\': 13.3, \'temp_f\': 55.9, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 8.1, \'wind_kph\': 13.0, \'wind_de